# Fundamentals of Accelerated Computing with CUDA C/C++

 Murilo Boratto$^1$

$^1$ SENAI CIMATEC <br />
     &nbsp;&nbsp;&nbsp; Supercomputing Center<br />

### Enabled GPU in Colab

**Go to Menu > Runtime > Change runtime > V100 GPU**

### Instalation the APIs OpenMP

In [ ]:
!sudo apt-get install libomp-dev

### Check if GPU is running or not, run the following command

In [1]:
!nvidia-smi

Wed Feb 19 17:41:07 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:60:00.0 Off |                    0 |
| N/A   42C    P0    44W / 300W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:61:00.0 Off |                    0 |
| N/A   

### Check if nvcc compiler is capable of using GPU

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


## `Matrix Multiply Code Portability and Optimization using CUDA`

### Sequential Code

In [3]:
%%writefile mm.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void kernel(int *A, int *B, int size)
{
 int i, j, k;

 for(i = 0; i < size; i++)
  for(j = 0; j < size; j++)
    for(k = 0; k < size; k++)
       B[i * size + j] += A[i * size + k] * B[k * size + j];
}

void initializeMatrix(int *matrix, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      matrix[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *matrix, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
  {
    for(j = 0; j < size; j++)
      printf("%d\t", matrix[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main (int argc, char **argv)
{
 int size = atoi(argv[1]);
 int i, j, k;
 double t1, t2;

 int  *A = (int *) malloc (sizeof(int) * size * size);
 int  *B = (int *) malloc (sizeof(int) * size * size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 t1 = omp_get_wtime();
   kernel(A, B, size);
 t2 = omp_get_wtime();

 printf("%d\t%f\n", size, t2-t1);

 //printMatrix(A,size);
 //printMatrix(B,size);

 // Free all our allocated memory
 free(A);
 free(B);

 return 0;

}

Overwriting mm.c


In [6]:
!gcc mm.c -o mm -fopenmp

In [7]:
!./mm 1000

1000	4.664137


### `CUDA Thread Hierarchy`

In [1]:
%%HTML

<div align="center">
<iframe src="https://docs.google.com/presentation/d/1J_GF6XACL0-Dk1BtJCeWiHwJCFcM_Hkx/edit?usp=share_link&ouid=117965215426975519312&rtpof=true&sd=true" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true">

</iframe></div>

In [20]:
%%writefile mm.cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

__global__ void kernel(int *A, int *B, int size)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  int k;

  if((i < size) && (j < size))
    for(k = 0; k < size; k++)
       B[i * size + j] += A[i * size + k] * B[k * size + j];

}

void initializeMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
       A[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
  {
    for(j = 0; j < size; j++)
       printf("%d\t", A[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{
  int size = atoi(argv[1]);
  int blockSize = atoi(argv[2]);
  double t1, t2;

  // Memory Allocation in the Host
  int  *A = (int *) malloc (sizeof(int) * size * size);
  int  *B = (int *) malloc (sizeof(int) * size * size);
 
  initializeMatrix(A, size);
  initializeMatrix(B, size);

  t1 = omp_get_wtime();
  
  // Memory Allocation in the Device
  int *d_A, *d_B;
  cudaMalloc((void **) &d_A, size * size * sizeof(int));
  cudaMalloc((void **) &d_B, size * size * sizeof(int));
 
  // Copy of data from host to device
  cudaMemcpy( d_A, A, size * size * sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy( d_B, B, size * size * sizeof(int), cudaMemcpyHostToDevice);
 
  // 2D Computational Grid
  dim3 dimGrid((int) ceil( (int) size / (int) blockSize ), (int) ceil( (int) size / (int) blockSize ));
  dim3 dimBlock( blockSize, blockSize);

       kernel<<<dimGrid, dimBlock>>>(A, B, size);

  // Copy of data from device to host
  cudaMemcpy( B, d_B, size * size * sizeof(int), cudaMemcpyDeviceToHost);

  t2 = omp_get_wtime();

  printf("%d\t%f\n", size, t2-t1);

 //printMatrix(B, size);
 
 // Memory Allocation in the Device
 cudaFree(d_A);
 cudaFree(d_B);
 
 // Memory Allocation in the Host
 free(A);
 free(B);

 return 0;

}

Overwriting mm.cu


In [21]:
!nvcc -arch=sm_75 mm.cu -o mm -Xcompiler -fopenmp

In [22]:
!./mm 1000 64

1000	0.191586


### `Grid-Stride Loops`

In [2]:
%%HTML

<div align="center">
<iframe src="https://docs.google.com/presentation/d/1tRO-HwqCfv8imhDO4S_8yAv8wEcJVttZ/edit?usp=sharing&ouid=117965215426975519312&rtpof=true&sd=true" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true">

</iframe></div>

In [24]:
%%writefile mm-gridStrideLoop.cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

__global__ void kernel(int *A, int *B, int size)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  int k;

  if((i < size) && (j < size))
    for(k = 0; k < size; k++)
       B[i * size + j] += A[i * size + k] * B[k * size + j];

}

__global__ void kernelGridStrideLoop(int *A, int *B, int size)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int idy = blockIdx.y * blockDim.y + threadIdx.y;
  int stride = gridDim.x * blockDim.x;
  int i, j, k;

  for(i = idx; i < size; i += stride)
    for(j = idy; j < size; j += stride)
    {
       for(k = 0; k < size; k++)
            B[i * size + j] += A[i * size + k] * B[k * size + j];
    }

}

void initializeMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      A[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
  {
    for(j = 0; j < size; j++)
      printf("%d\t", A[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{
  int size = atoi(argv[1]);
  int blockSize = atoi(argv[2]);
  double t1, t2;

  // Memory Allocation in the Host
  int  *A = (int *) malloc (sizeof(int) * size * size);
  int  *B = (int *) malloc (sizeof(int) * size * size);
 
  initializeMatrix(A, size);
  initializeMatrix(B, size);

  t1 = omp_get_wtime();
    
  // Memory Allocation in the Device
  int *d_A, *d_B;
  cudaMalloc((void **) &d_A, size * size * sizeof(int));
  cudaMalloc((void **) &d_B, size * size * sizeof(int));
  
  // Copy of data from host to device
  cudaMemcpy( d_A, A, size * size * sizeof(int), cudaMemcpyHostToDevice );
  cudaMemcpy( d_B, B, size * size * sizeof(int), cudaMemcpyHostToDevice );
  
  // 2D Computational Grid
  dim3 dimGrid( (int) ceil( (int) size / (int) blockSize ), (int) ceil( (int) size / (int) blockSize ) );
  dim3 dimBlock( blockSize, blockSize);

            kernelGridStrideLoop<<<dimGrid, dimBlock>>>(A, B, size);

  // Copy of data from device to host
  cudaMemcpy( B, d_B, size * size * sizeof(int), cudaMemcpyDeviceToHost );

  t2 = omp_get_wtime();

  printf("%d\t%f\n", size, t2-t1);

 //printMatrix(A, size);
 //printMatrix(B, size);

 // Memory Allocation in the Device
 cudaFree(d_A);
 cudaFree(d_B);
 
 // Memory Allocation in the Host
 free(A);
 free(B);
 
 return 0;
}

Overwriting mm-gridStrideLoop.cu


In [25]:
!nvcc -arch=sm_75 mm-gridStrideLoop.cu -o mm-gridStrideLoop -Xcompiler -fopenmp

In [27]:
!./mm-gridStrideLoop 1000 64

1000	0.190511


### `Unified Memory (cudaMallocManaged)`

In [3]:
%%HTML

<div align="center">
<iframe src="https://docs.google.com/presentation/d/1ZgEGCivfxKS6DDHsq1-3-k4YELQQknZ0/edit?usp=share_link&ouid=117965215426975519312&rtpof=true&sd=true" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true">

</iframe></div>

In [41]:
%%writefile mm-cudaMallocManaged.cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

__global__ void kernel(int *A, int *B, int size)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  int k;

  if((i < size) && (j < size))
     for(k = 0; k < size; k++)
        B[i * size + j] += A[i * size + k] * B[k * size + j];

}

__global__ void kernelGridStrideLoop(int *A, int *B, int size)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int idy = blockIdx.y * blockDim.y + threadIdx.y;
  int stride = gridDim.x * blockDim.x;
  int i, j, k;

  for(i = idx; i < size; i += stride)
    for(j = idy; j < size; j += stride)
    {
       for(k = 0; k < size; k++)
            B[i * size + j] += A[i * size + k] * B[k * size + j];
    }

}

void initializeMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      A[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
  {
    for(j = 0; j < size; j++)
      printf("%d\t", A[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main(int argc, char **argv)
{ 
 int size = atoi(argv[1]);
 int blockSize = atoi(argv[2]); ;
 double t1, t2;
 int *A,  *B;

 t1 = omp_get_wtime();

 cudaMallocManaged(&A, sizeof(int) * size * size);
 cudaMallocManaged(&B, sizeof(int) * size * size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 dim3 dimGrid( (int) ceil( (int) size / (int) blockSize ), (int) ceil( (int) size / (int) blockSize ) );
 dim3 dimBlock( blockSize, blockSize);

      kernelGridStrideLoop<<<dimGrid, dimBlock>>>(A, B, size);
      cudaDeviceSynchronize();

 t2 = omp_get_wtime();

printf("%d\t%f\n", size, (t2-t1));

//printMatrix(A, size);
//printMatrix(B, size);

// Free all our allocated memory
cudaFree(A);
cudaFree(B);

return 0;

}


Overwriting mm-cudaMallocManaged.cu


In [42]:
!nvcc mm-cudaMallocManaged.cu -o mm-cudaMallocManaged -Xcompiler -fopenmp

In [43]:
!./mm-cudaMallocManaged 1000 64

1000	0.222651


#### `Streaming Multiprocessors (SMs)`

In [4]:
%%HTML

<div align="center">
<iframe src="https://docs.google.com/presentation/d/18z3x55kxCCjGZ3LVKOtSN5q8qXe4swFL/edit?usp=sharing&ouid=117965215426975519312&rtpof=true&sd=true" frameborder="0" width="900" height="550" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true">

</iframe></div>

In [49]:
%%writefile mm-streamingMultiprocessors.cu
#include <cuda.h>
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

__global__ void kernelGridStrideLoop(int *A, int *B, int size)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  int idy = blockIdx.y * blockDim.y + threadIdx.y;
  int stride = gridDim.x * blockDim.x;
  int i, j, k;

  for(i = idx; i < size; i += stride)
    for(j = idy; j < size; j += stride)
    {
       for(k = 0; k < size; k++)
         B[i * size + j] += A[i * size + k] * B[k * size + j];
    }

}

void initializeMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      A[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *A, int size)
{
  int i, j;

  for(i = 0; i < size; i++)
  {
    for(j = 0; j < size; j++)
      printf("%d\t", A[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main (int argc, char **argv)
{
 int size = atoi(argv[1]);
 int sizeblock = atoi(argv[2]); ;
 double t1, t2;
 int *A,  *B;

 t1 = omp_get_wtime();

 cudaMallocManaged(&A, sizeof(int) * size * size);
 cudaMallocManaged(&B, sizeof(int) * size * size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 int deviceId, numberOfSMs;
 cudaGetDevice(&deviceId);
 cudaDeviceGetAttribute(&numberOfSMs, cudaDevAttrMultiProcessorCount, deviceId);

 int NUMBER_OF_BLOCKS = numberOfSMs * 32;
 int NUMBER_OF_THREADS = 1024;

      kernelGridStrideLoop<<< NUMBER_OF_BLOCKS, NUMBER_OF_THREADS>>>(A, B, size);
      cudaDeviceSynchronize();

 t2 = omp_get_wtime();

 printf("%d\t%f\n", size, t2-t1);

//printMatrix(B, size);

// Free all our allocated memory
 cudaFree(A);
 cudaFree(B);

 return 0;
}

Overwriting mm-streamingMultiprocessors.cu


In [45]:
!nvcc mm-streamingMultiprocessors.cu -o mm-streamingMultiprocessors -Xcompiler -fopenmp

In [47]:
!./mm-streamingMultiprocessors 1000 64

1000	0.226932


## `Profilling GPU core`

The GPU has many units working in parallel, and it is common for it to be bound by different units at different frame sequences. Due to the nature of this behavior, it is beneficial to identify where the GPU cost is going when searching for bottlenecks and to understand what a GPU bottleneck is. Some applications help developers identify bottlenecks, which is useful when optimizing performance, following some NVIDIA profilling tools.

In [59]:
%%writefile vector-add.cu
#include <stdio.h>
#include <cuda.h>

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
    a[i] = num;
  
}

__global__ void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
    result[i] = a[i] + b[i];
}

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main(int argc, char **argv)
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  int deviceId;
  cudaGetDevice(&deviceId);

  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId);
  int multiProcessorCount = props.multiProcessorCount;
  threadsPerBlock = 1024;
  numberOfBlocks = 32 * multiProcessorCount;
  
  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  checkElementsAre(7, c, N);

  // Free all our allocated memory  
  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
    
  return 0;      
}

Overwriting vector-add.cu


In [60]:
!nvcc vector-add.cu -o vector-add

### ⊗ NSYS

`NVIDIA Nsight Systems` (nsys) is a system-wide performance analysis tool designed to visualize an application’s algorithms, help you identify the largest opportunities to optimize, and tune to scale efficiently across any quantity or size of GPUs.

The command `nsys profile` will generate a `qdrep` report file which can be used in a variety of manners. We use the `--stats=true` flag here to indicate we would like summary statistics printed. There is quite a lot of information printed:

- Profile configuration details
- Report file(s) generation details
- **CUDA API Statistics**
- **CUDA Kernel Statistics**
- **CUDA Memory Operation Statistics (time and size)**
- OS Runtime API Statistics

In this lab you will primarily be using the nsys im command line. In the next, you will be using the generated report files to give to the Nsight Systems GUI for visual profilling.

In [61]:
!nsys profile --stats=true ./vector-add

Success! All values calculated correctly.
Generating '/tmp/nsys-report-2f5d.qdstrm'
[1/8] [========================100%] report4.nsys-rep
[2/8] [========================100%] report4.sqlite
[3/8] Executing 'nvtxsum' stats report
SKIPPED: /home/murilo/report4.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrtsum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)   Max (ns)    StdDev (ns)        Name     
 --------  ---------------  ---------  ------------  ------------  --------  -----------  ------------  --------------
     81,0    1.075.978.685         66  16.302.707,0  10.146.761,0     1.834  100.438.884  24.925.543,0  poll          
     10,0      137.811.825         59   2.335.793,0   2.124.556,0    16.228   21.238.331   3.502.350,0  sem_timedwait 
      5,0       77.532.986      1.032      75.128,0      19.884,0     1.045   26.221.327     832.433,0  ioctl         
      2,0       26.460.687         33     801.839,

After profilling the application, answer the following questions using information displayed in the `CUDA Kernel Statistics` section of the profilling output.

### ⊗ NCU

`NVIDIA Nsight Compute` (ncu) provides a non-interactive way to profile applications from the command line. It can print the results directly on the command line or store them in a report file. 

To print profilling information on the command line on the NCU, do not specify the output file (flag -o). Or, if you want to generate the output file (-o) and still see it on the command line, you can use the --page flag.

In [62]:
!ncu --set detailed vector-add

==PROF== Connected to process 36723 (/home/murilo/vector-add)
==PROF== Profiling "addVectorsInto" - 0: 0%....50%....100% - 24 passes
Success! All values calculated correctly.
==PROF== Disconnected from process 36723
[36723] vector-add@127.0.0.1
  addVectorsInto(float *, float *, float *, int), 2025-Feb-19 18:56:49, Context 1, Stream 7
    Section: GPU Speed Of Light Throughput
    ---------------------------------------------------------------------- --------------- ------------------------------
    DRAM Frequency                                                           cycle/usecond                         852,16
    SM Frequency                                                             cycle/nsecond                           1,26
    Elapsed Cycles                                                                   cycle                        809.889
    Memory [%]                                                                           %                          71,24
    DRAM T

## `Concurrent CUDA Streams`

In CUDA programming, a **stream** is a series of commands that execute in order. In CUDA applications, kernel execution, as well as some memory transfers, occur within CUDA streams. Up until this point in time, you have not been interacting explicitly with CUDA streams, but in fact, your CUDA code has been executing its kernels inside of a stream called *the default stream*. CUDA programmers can create and utilize non-default CUDA streams in addition to the default stream, and in doing so, perform multiple operations, such as executing multiple kernels, concurrently, in different streams. Using multiple streams can add an additional layer of parallelization to your accelerated applications, and offers many more opportunities for application optimization.

### Creating, Utilizing, and Destroying Non-Default CUDA Streams

The following code snippet demonstrates how to create, utilize, and destroy a non-default CUDA stream. You will note, that to launch a CUDA kernel in a non-default CUDA stream, the stream must be passed as the optional 4th argument of the execution configuration. Up until now you have only utilized the first 2 arguments of the execution configuration:

```cpp
cudaStream_t stream;       // CUDA streams are of type `cudaStream_t`.
cudaStreamCreate(&stream); // Note that a pointer must be passed to `cudaCreateStream`.

someKernel<<<number_of_blocks, threads_per_block, 0, stream>>>(); // `stream` is passed as 4th EC argument.

cudaStreamDestroy(stream); // Note that a value, not a pointer, is passed to `cudaDestroyStream`.
```

Outside the scope of this lab, but worth mentioning, is the optional 3rd argument of the execution configuration. This argument allows programmers to supply the number of bytes in **shared memory** (an advanced topic that will not be covered presently) to be dynamically allocated per block for this kernel launch. The default number of bytes allocated to shared memory per block is `0`, and for the remainder of the lab, you will be passing `0` as this value, in order to expose the 4th argument.

In [63]:
%%writefile print-numbers-solution.cu
#include <stdio.h>
#include <unistd.h>

__global__ void printNumber(int number)
{
  printf("%d\n", number);
}

int main(int argc, char **argv)
{
  int i;
    
  for(i = 0; i < 5; ++i)
  {
    cudaStream_t stream;
    cudaStreamCreate(&stream);
    printNumber<<<1, 1, 0, stream>>>(i);
    cudaStreamDestroy(stream);
  }
    
  cudaDeviceSynchronize();
    
  return 0;  
}

Overwriting print-numbers-solution.cu


In [67]:
!nvcc print-numbers-solution.cu -o print-numbers-solution

In [65]:
!./print-numbers-solution

0
1
2
3
4


## `Asynchronous Memory Prefetching`

Prefetching also tends to migrate data in larger chunks, and therefore fewer trips, than on-demand migration. This makes it an excellent fit when data access needs are known before runtime, and when data access patterns are not sparse.

CUDA Makes asynchronously prefetching managed memory to either a GPU device or the CPU easy with its `cudaMemPrefetchAsync` function. Here is an example of using it to both prefetch data to the currently active GPU device, and then, to the CPU:

```cpp

int deviceId;
cudaGetDevice(&deviceId);                                         // The ID of the currently active GPU device.

cudaMemPrefetchAsync(pointerToSomeUMData, size, deviceId);        // Prefetch to GPU device.
cudaMemPrefetchAsync(pointerToSomeUMData, size, cudaCpuDeviceId); // Prefetch to host. `cudaCpuDeviceId` is a
                                                                  // built-in CUDA variable.

```

In [72]:
%%writefile vector-add-prefetching.cu
#include <stdio.h>
#define N 2048 * 2048 // Number of elements in each vector

__global__ void saxpy(int * a, int * b, int * c)
{
  // Determine our unique global thread ID, so we know which element to process
  int tid = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  
  for (int i = tid; i < N; i += stride)
     c[i] = 2 * a[i] + b[i];
}

int main(int argc, char **argv)
{
  int *a, *b, *c;

  int size = N * sizeof (int); // The total number of bytes per vector

  int deviceId;
  int numberOfSMs;

  cudaGetDevice(&deviceId);
  cudaDeviceGetAttribute(&numberOfSMs, cudaDevAttrMultiProcessorCount, deviceId);

  // Allocate memory
  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  // Initialize memory
  for(int i = 0; i < N; ++i )
  {
    a[i] = 2;
    b[i] = 1;
    c[i] = 0;
  }

  cudaMemPrefetchAsync(a, size, deviceId);
  cudaMemPrefetchAsync(b, size, deviceId);
  cudaMemPrefetchAsync(c, size, deviceId);

  int threads_per_block = 256;
  int number_of_blocks = numberOfSMs * 32;

  saxpy <<<number_of_blocks, threads_per_block>>>( a, b, c );

  cudaDeviceSynchronize(); // Wait for the GPU to finish

  // Print out the first and last 5 values of c for a quality check
  for( int i = 0; i < 5; ++i )
    printf("c[%d] = %d, ", i, c[i]);
  printf ("\n");
  for( int i = N-5; i < N; ++i )
    printf("c[%d] = %d, ", i, c[i]);
  printf ("\n");

  // Free all our allocated memory
  cudaFree(a); 
  cudaFree(b); 
  cudaFree(c);
    
  return 0;  
}

Overwriting vector-add-prefetching.cu


In [69]:
!nvcc vector-add-prefetching.cu -o vector-add-prefetching

In [70]:
!./vector-add-prefetching

c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 
